# TM10007 Assignment template

In [3]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [25]:
#Load packages

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import datasets as ds
from sklearn import metrics
from sklearn import model_selection


# Classifiers --> later bijwerken welke we gebruiken
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


# Importing Data loading functions. Uncomment the one you want to use
from worcliver.load_data import load_data

from scipy.stats import ttest_ind, mannwhitneyu, shapiro
from statsmodels.stats.multitest import multipletests

In [26]:
# Loading the data
data = load_data()

# Presentation of the data:

In [60]:
# Description of the data
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

# Display basic info
print("First five rows of the dataset:")
print(data.head(), "\n")

print("Dataset info:")
print(data.info(), "\n")

print("Summary statistics:")
print(data.describe(), "\n")

# Counting missing values
missing_values = data.isnull().sum()
total_missing = missing_values.sum()

print(f"Total missing values in the dataset: {total_missing}\n")
print(f"Missing values per column:\n{missing_values[missing_values > 0]}\n")

# Counting categorical and numerical columns
categorical_columns = data.select_dtypes(include=['object']).columns
numerical_columns = data.select_dtypes(exclude=['object']).columns

print(f"Number of categorical columns: {len(categorical_columns)}")
print(f"Number of numerical columns: {len(numerical_columns)}\n")

# Count of each label (benign/malignant)
label_counts = data['label'].value_counts()

print("Label distribution:")
print(label_counts)


The number of samples: 186
The number of columns: 494
First five rows of the dataset:
                 label  PREDICT_original_sf_compactness_avg_2.5D  \
ID                                                                 
Liver-001_0     benign                                  0.878471   
Liver-002_0     benign                                  0.878945   
Liver-003_0     benign                                  0.766162   
Liver-004_0  malignant                                  0.825737   
Liver-005_0  malignant                                  0.828831   

             PREDICT_original_sf_compactness_std_2.5D  \
ID                                                      
Liver-001_0                                  0.023468   
Liver-002_0                                  0.039922   
Liver-003_0                                  0.064140   
Liver-004_0                                  0.062047   
Liver-005_0                                  0.062635   

             PREDICT_original_sf_rad_

# Changing labels malignant and benign to 0 and 1

In [68]:
#changing string (malignant or benign) to 0 or 1
num_data = data.copy()

# Transform labels: benign -> 1, malignant -> 0
num_data['label'] = num_data['label'].map({'benign': 1, 'malignant': 0})

# Splitting data in train and test test

In [75]:
#Splitting data in test and train set 
y = num_data['label']
x = num_data[:]

#x is features, y  = maligne / benign
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

#print the shape of the data sets  
print(f'The shape of the train data: {x_train.shape}')
print(f'The shape of the test data: {x_test.shape}')

x_train.to_csv('x_train.csv', index=False) 
x_test.to_csv('x_test.csv', index=False) 


The shape of the train data: (148, 494)
The shape of the test data: (38, 494)


# Selecting significant features

In [98]:
# Select significant features
# Add the label column back to x_train
x_train_with_label = x_train.copy()
x_train_with_label['label'] = y_train

# Separate benign and malignant samples in training data
benign = x_train_with_label[x_train_with_label['label'] == 1]
malignant = x_train_with_label[x_train_with_label['label'] == 0]

x_train_with_label.to_csv('x_train_label.csv', index=False) 
# Perform statistical analysis
features = []
sig_features = []

# Loop through all features except 'label'
for feature in x_train_with_label.columns:
    if feature == 'label':  # Skip the label column
        continue
        
    # Data for current feature
    benign_values = benign[feature].dropna()
    malignant_values = malignant[feature].dropna()
    
    # Normality test (Shapiro-Wilk test, p < 0.05 means not normally distributed)
    _, p_benign = shapiro(benign_values)
    _, p_malignant = shapiro(malignant_values)
    
    # Determine which test to use
    if p_benign > 0.05 and p_malignant > 0.05:  # Both distributions are normal
        test_type = "t-test"
        stat, p_value = ttest_ind(benign_values, malignant_values, equal_var=False)  # Welch's t-test
    else:
        test_type = "Mann-Whitney U-test"
        stat, p_value = mannwhitneyu(benign_values, malignant_values, alternative='two-sided')
    
    # Save results
    feature_entry = {
        "Feature": feature,
        "Test": test_type,
        "p_value": p_value
    }
    features.append(feature_entry)
    
    # Check for significance (p <= 0.05)
    if p_value <= 0.05:
        sig_entry = feature_entry.copy()
        # sig_entry['benign_mean'] = benign_values.mean()
        # sig_entry['malignant_mean'] = malignant_values.mean()
        # sig_entry['benign_std'] = benign_values.std()
        # sig_entry['malignant_std'] = malignant_values.std()
        sig_features.append(sig_entry)

# Convert to DataFrames
features_df = pd.DataFrame(features)
sig_features_df = pd.DataFrame(sig_features)

# Multiple testing correction (False Discovery Rate - Benjamini-Hochberg)
if not features_df.empty:
    reject, p_corrected, alphacSidak, alphacBonf = multipletests(features_df["p_value"], method='fdr_bh')
    features_df["p_value_corrected"] = p_corrected

# Sort significant features by p-value only if there are significant features
if not sig_features_df.empty:
    sig_features_df = sig_features_df.sort_values(by="p_value")

# Print and save results
# print("\nTotal Significant Features:")
# print(sig_features_df)
print(f"\nNumber of significant features: {len(sig_features_df)}")

# Get the list of significant feature names
sig_feature_names = sig_features_df['Feature'].tolist() if not sig_features_df.empty else []

# Select only significant features for training data if there are any
if sig_feature_names:
    x_train_sig = x_train[sig_feature_names].copy()
    # Save results to CSV
    x_train_sig.to_csv('x_train_sig.csv', index=False)
else:
    print("No significant features found. Cannot create x_train_sig.")

# Save results to CSV
# features_df.to_csv('features.csv', index=False)
sig_features_df.to_csv('sig_features_df.csv', index=False)

c:\Users\ellen\Nieuwe_map\envs\KT-2023\lib\site-packages\scipy\stats\morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "



Number of significant features: 154


# Linear classifier

In [ ]:
# Linear classifier

# Test how many feautures are normal distributed

#From features_df, sum how many t test used, i.e. are normal distributed from all features
normal_distributed = features_df.loc[features_df['Test'] == 't-test'].count() 
print(f' The ammount of normal distributed features is: {normal_distributed}')
print(f' Percentages of normal distributed features: {normal_distributed/x_train.shape[0]*100}')


#the ammount of normal distibuted from significant features 
normal_distributed_sig = sig_features_df.loc[features_df['Test'] == 't-test'].count() 
print(f' The ammount of normal distributed features is: {normal_distributed_sig}')

#hier wordt de lineaire classif gemaakt, bepalen of we dat willen met 52
#lda = LinearDiscriminantAnalysis()
#lda = lda.fit(x_train, y_train)
#y_pred = lda.predict(x_test)


 The ammount of normal distributed features is: Feature              77
Test                 77
p_value              62
p_value_corrected     0
normaal_verdeeld     77
dtype: int64
 Percentages of normal distributed features: Feature              52.027027
Test                 52.027027
p_value              41.891892
p_value_corrected     0.000000
normaal_verdeeld     52.027027
dtype: float64
 The ammount of normal distributed features is: Feature    38
Test       38
p_value    38
dtype: int64


# Quadratic classifier

In [86]:
# Quadratic classifier
q_clas = QuadraticDiscriminantAnalysis()
q_clas = q_clas.fit(x_train, y_train)

q_clas_pred = q_clas.predict(x_test)

c:\Users\ellen\Nieuwe_map\envs\KT-2023\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


# Random forrest classifier

In [91]:
# random forrest code
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', 
                                              max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                                              min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, 
                                              min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, 
                                              random_state=None, verbose=0, warm_start=False, class_weight=None, 
                                              ccp_alpha=0.0, max_samples=None, monotonic_cst=None)


SyntaxError: invalid syntax (2624179345.py, line 5)

# Calculate mean and std of individual classes

In [ ]:
#Decription 
## Dit nog aanpassen naar berekenen op basis van 'label' ipv deze nieuwe csv files
# Mean
benign_mean = benign.mean(numeric_only=True)
malignant_mean = malignant.mean(numeric_only=True)

# Standard deviation
benign_std = benign.std(numeric_only=True)
malignant_std = malignant.std(numeric_only=True)

#Print statistics
benign_stats = pd.DataFrame({
    'mean': benign.mean(numeric_only=True),
    'std': benign.std(numeric_only=True)
})
print(benign_stats)
benign_stats.T.to_csv('benign_stats.csv')


malignant_stats = pd.DataFrame({
    'mean': malignant.mean(numeric_only=True),
    'std': malignant.std(numeric_only=True)
})
malignant_stats.T.to_csv('malignant_stats.csv')